# Pantheon SN 1A analysis

## Top level imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import integrate
import os
from scipy.optimize import curve_fit
from IPython.display import display, Math
from matplotlib.patches import Ellipse

## Fisher Matrix Functions

In [2]:
def r(z,params):
    '''
    params: tuple or array, containing [omega_m,w]
    '''
    return (1/H_0) * integrate.quad(r_func,0,z,args=(params))[0]

def r_func(z,omega_m,w):
    omega_de = 1-omega_m
    return 1/np.sqrt(omega_m*(1+z)**3+omega_de*(1+z)**(3*(1+w)))

def h(z,params):
    return integrate.quad(h_func,z,0,args=(params))[0]

def h_func(z,omega_m,w):
    return (1+z)**3 * ((1+z)**(3*w)-1)/(omega_m*(1+z)**3+(1-omega_m)*(1+z)**(3*(1+w)))**(3/2)

def g(z,params):
    return integrate.quad(g_func,z,0,args=(params))[0]

def g_func(z,omega_m,w):
    omega_de = 1-omega_m
    return omega_de*(1+z)**(3*(1+w))*np.log(1+z)/(omega_m*(1+z)**3 + (1-omega_m)*(1+z)**(3*(1+w)))**(3/2)

# def del_m_del_w(z,params):
#     return 15/(2*np.log(10)) * g(z,params)/r(z,params)

# def del_m_del_omega_m(z,params):
#     return 2.5/np.log(10) * h(z,params)/r(z,params)


def del_m_del_w(z,params):
    #return 15/(2*np.log(10)) * g(z,params)/r(z,params)
    dw=0.01
    return 5/(np.log(10)*r_func(z,omega_m,w)) *(r_func(z,omega_m,w+dw)-r_func(z,omega_m,w-dw))/(2*dw)

def del_m_del_omega_m(z,params):
    #return 2.5/np.log(10) * h(z,params)/r(z,params)
    dOm=0.01
    return 5/(np.log(10)*r_func(z,omega_m,w)) *(r_func(z,omega_m+dOm,w)-r_func(z,omega_m-dOm,w))/(2*dOm)

## Importing data

In [3]:
def importDF(path,col_ignore=False,cols=None,sep=","):
    df = pd.read_csv(path,sep=sep)
    if col_ignore:
        df = df[cols]
    return df

def dropArr():
    return ['zcmb','mb','dmb']

In [4]:
path = "/Users/sean/Desktop/Cosmo proj/lcparam_full_long_zhel.txt" # Path to the data file
df = importDF(path,col_ignore=True,cols=dropArr(),sep=" ")

In [5]:
systematics_path = "/Users/sean/Desktop/Cosmo proj/sys_full_long.txt"
systematics = np.loadtxt(systematics_path,skiprows=1)
covariance = np.reshape(systematics,(int(np.sqrt(len(systematics))),int(np.sqrt(len(systematics)))))

for k in range(len(covariance)):
    covariance[k][k] = covariance[k][k] + df['dmb'][k]**2

## Fisher Matrix calculations

In [6]:
def generate_fisher(nparams,uncertainty,redshifts,del_funcs,params):
    '''
    nparams: number of parameters
    uncertainty: 2d array of magnitude covariances
    redshifts: array of measured redshifts
    del_funcs: array that contains all of the derivative of magnitude functions - must be nparams*nparams in size
    params: tuple that contains (omega_m,w)
    '''
    fisher = np.empty((nparams,nparams))
    for k in range(nparams):
        for l in range(nparams):
            all_delta_m_k = np.zeros(len(redshifts))
            all_delta_m_l = np.zeros(len(redshifts))
            for entry in range(len(redshifts)):
                all_delta_m_k[entry] = del_funcs[k](redshifts[entry],params)
                all_delta_m_l[entry] = del_funcs[l](redshifts[entry],params)
            fisher[k][l] = np.matmul(np.matmul(all_delta_m_k.T,np.linalg.inv(uncertainty)), all_delta_m_l)
    return fisher

In [7]:
omega_m,w = 0.3,-1

h_little=1
H_0 = 100*h_little

Fisher = generate_fisher(2,covariance,df['zcmb'],[del_m_del_omega_m,del_m_del_w],(omega_m,w))

In [8]:
sigma_txt = [r'\sigma(\Omega_m) \geq',r'\sigma(w) \geq']

marginalized = np.zeros(len(Fisher))
unmarginalized = np.zeros(len(Fisher))

for k in range(len(Fisher)):
    marginalized[k] = np.sqrt(np.linalg.inv(Fisher)[k][k])
    unmarginalized[k] = 1/np.sqrt(Fisher[k][k])
    
    print("Marginalized over other parameters")
    display(Math(sigma_txt[k]+"{:1.2E}".format(marginalized[k])))
    print("Unmarginalized, other parameters fixed and known")
    display(Math(sigma_txt[k]+"{:1.2E}".format(unmarginalized[k])))

Marginalized over other parameters


<IPython.core.display.Math object>

Unmarginalized, other parameters fixed and known


<IPython.core.display.Math object>

Marginalized over other parameters


<IPython.core.display.Math object>

Unmarginalized, other parameters fixed and known


<IPython.core.display.Math object>

### A few functions for calculations

In [9]:
def chi_squared(ydata,yerr,xdata,params,model):
    A,B,C = 0,0,0
    for j in range(len(ydata)):
        A+=((ydata[j]-mag_theoretical(xdata[j],params))/yerr[j])**2
        B+=(ydata[j]-mag_theoretical(xdata[j],params))/yerr[j]**2
        C+=1/yerr[j]**2
    return A-B**2/C

def chi_squared_cov(ydata,cov,xdata,params,model):
    arr = np.empty(len(ydata))
    
    for l in range(len(arr)):
        arr[l] = ydata[l]-model(xdata[l] , params)

    Identity = np.ones(len(arr)) # Define identity column vector here

    A = np.matmul(np.matmul(arr.T,np.linalg.inv(cov)),arr)
    B = np.matmul(np.matmul(Identity.T,np.linalg.inv(cov)),arr)
    C = np.matmul(np.matmul(Identity.T,np.linalg.inv(cov)),Identity)
    
    return A-B**2/C

def mag_theoretical(z,params):
    return 5*np.log10(H_0*luminosity_distance(z,params))

def luminosity_distance(z,params):
    return (1+z)*r(z,params)

## The PANTHEON measurements

In [10]:
pantheon_w, pantheon_w_sigma = -1.026,0.41
pantheon_omega_m,pantheon_omega_m_sigma = 0.307,0.012

# Pulled from https://arxiv.org/abs/1710.00845

## Setting the parameters of the grid

In [11]:
d_omega_m = 0.005
d_w = 0.01

omega_m_min,omega_m_max = 0.2,0.5
w_min,w_max = -1.5,-0.5

best_fit_chi_lambda_fixed_omega_m = 1E10
best_fit_chi_lambda_fixed_w = 1E10
best_fit_chi_wcdm = 1E10

fiducial_omega_m = 0.3
fiducial_w = -1

omega_m_arr = np.arange(omega_m_min,omega_m_max+d_omega_m, step=d_omega_m)
w_arr = np.arange(w_min,w_max+d_w,step=d_w)

In [12]:
covariance_flag = True

if covariance_flag:
    chi_func = chi_squared_cov
    unc = covariance
    end_string = 'with_covariance.jpg'
else:
    chi_func = chi_squared
    end_string = 'no_covariance.jpg'
    unc = df['dmb']

## Cycling over the grid

In [ ]:
chi_arr = np.empty((len(omega_m_arr),len(w_arr)),dtype=float)
chi_arr_fixed_w = np.empty(len(omega_m_arr))
chi_arr_fixed_omega_m = np.empty(len(w_arr))

omega_m_iter,w_iter = 0,0

for omega_m in omega_m_arr:
    print("omega_m =","{:.3f}".format(omega_m),end=". ")
    for w in w_arr:
        
        chi_val = chi_func(df['mb'],unc,df['zcmb'],(omega_m,w),mag_theoretical)
        chi_arr[omega_m_iter][w_iter] = chi_val
        
        if chi_val<best_fit_chi_wcdm:
            best_fit_chi_wcdm,best_omega_m_wcdm,best_w_wcdm = chi_val,omega_m,w
        if "{:.5f}".format(w)=="{:.5f}".format(fiducial_w): # If lambda CDM cosmology
            chi_arr_fixed_w[omega_m_iter] = chi_val
            if chi_val<best_fit_chi_lambda_fixed_w:
                best_fit_chi_lambda_fixed_w,best_omega_m_lambda = chi_val,omega_m
                
        if "{:.5f}".format(omega_m) == "{:.5f}".format(fiducial_omega_m):
            chi_arr_fixed_omega_m[w_iter] = chi_val
            if chi_val<best_fit_chi_lambda_fixed_omega_m:
                best_fit_chi_lambda_fixed_omega_m,best_w_lambda = chi_val,w
                
        w_iter+=1
    omega_m_iter+=1
    w_iter = 0
    print("Testing of parameters complete")

omega_m = 0.200. Testing of parameters complete
omega_m = 0.205. Testing of parameters complete
omega_m = 0.210. 

In [ ]:
print("---"*7,"wCDM","---"*7)
print(r"Omega_m that minimizes chi^2 in w-CDM:","{:0.4f}".format(best_omega_m_wcdm))
print(r"w that minimizes chi^2 in w-CDM:","{:0.4f}".format(best_w_wcdm))
print(r"$\chi^2$ of best fit Omega_m in w-CDM:","{:0.4f}".format(best_fit_chi_wcdm),end='\n\n')

print("---"*4,"Lambda-CDM, fixed w={:.4f}".format(fiducial_w),"---"*4)
print(r"Omega_m that minimizes chi^2 in lambda CDM with fixed w:","{:0.4f}".format(best_omega_m_lambda))
print(r"chi^2 of best fit Omega_m in lambda CDM:","{:0.4f}".format(best_fit_chi_lambda_fixed_w),end='\n\n')

print("---"*3,"Lambda-CDM, with omega_m={:.4f}".format(fiducial_omega_m),"---"*3)
print(r"w that minimizes chi^2 in lambda CDM with omega_m fixed:","{:0.4f}".format(best_w_lambda))
print(r"chi^2 of best fit w in lambda CDM:","{:0.4f}".format(best_fit_chi_lambda_fixed_omega_m),end='\n\n')

print("---"*4,"PANTHEON Measurements","---"*4)
print(r"PANTHEON Omega_m:","{:0.4f}".format(pantheon_omega_m))
print(r"PANTHEON w:","{:0.4f}".format(pantheon_w))

## Manipulating the chi squared values into likelihood values

In [ ]:
likelihood_arr = np.exp(-chi_arr/2 + 520).T
likelihood_arr = likelihood_arr/np.max(likelihood_arr)

likelihood_arr_fixed_w = np.exp(-chi_arr_fixed_w/2 + 520).T
likelihood_arr_fixed_w = likelihood_arr_fixed_w/np.max(likelihood_arr_fixed_w)

likelihood_arr_fixed_omega_m = np.exp(-chi_arr_fixed_omega_m/2 + 520).T
likelihood_arr_fixed_omega_m = likelihood_arr_fixed_omega_m/np.max(likelihood_arr_fixed_omega_m)

## Two helper functions

In [ ]:
def find_level(level,arr):
    '''
    Find the value that encloses x% of the total weight
    '''
    sum = np.sum(arr)
    
    target = level*sum

    one_d_arr = np.hstack(likelihood_arr)
    sorted_arr = np.sort(one_d_arr)[::-1]
    
    val=0
    iter = 0

    while val<target:
        last = sorted_arr[iter]
        val+=last
        iter+=1

    return last

def comparison(measurement,fiducial,sigma):
    return np.abs(measurement - fiducial)/sigma

# $\Omega_m$ - w plane

In [ ]:
# omega_m_fisher_high, omega_m_fisher_low = best_omega_m_wcdm + marginalized[0],best_omega_m_wcdm - marginalized[0]
# w_fisher_high, w_fisher_low = best_w_wcdm + marginalized[1],best_w_wcdm - marginalized[1]

In [ ]:
fig,ax = plt.subplots(figsize=[8,6])

im1 = ax.contourf(omega_m_arr,w_arr,likelihood_arr,vmin=0,vmax=1,levels=np.arange(0,1.01,step=0.01),cmap='cividis')
ax.set_xlabel(r'$\Omega_m$')
ax.set_ylabel('w')
ax.set_title(r"Likelihood over $\Omega_m$-w plane")
ax.set_xticks(np.arange(min(omega_m_arr),max(omega_m_arr)+d_omega_m,0.02))
plt.colorbar(im1,label="Likelihood",ticks=np.arange(0,1.1,step=0.1))

ax.scatter(best_omega_m_wcdm,best_w_wcdm,marker="+",color='black',label=r"My max($L(\Omega_m, w$)) = ("+"{:.3f}, ".format(best_omega_m_wcdm)+"{:.3f}".format(best_w_wcdm)+")")

sigmas = [0.6827,0.9545,0.997300, 0.999936, 	0.999999 ,	0.999999998,0.999999999997]
iter=0
for level in sigmas[0:3]:
    level = find_level(level,likelihood_arr)
    label = str(iter+1)+r"$\sigma$"
    CS = ax.contour(omega_m_arr,w_arr,likelihood_arr, levels=[level],cmap='Oranges')
    ax.clabel(CS, inline=True, fontsize=10,fmt=label)
    iter+=1

ax.scatter(pantheon_omega_m,pantheon_w,marker="x",color='r',
           label=r"Pantheon max($L(\Omega_m, w$)) = ("+"{:.3f}, ".format(pantheon_omega_m)+"{:.3f}".format(pantheon_w)+")")

ellipse = Ellipse(xy=(best_omega_m_wcdm,best_w_wcdm), width=marginalized[0], height=marginalized[1], 
                        edgecolor='green', fc='None', ls='--',lw=2,label = r"Fisher 1-$\sigma$ limits")
ax.add_patch(ellipse)

plt.legend()

fig.tight_layout()

plt.savefig(os.getcwd()+"/TwoDimLikelihood_"+end_string,dpi=180)

plt.show()

### For w=-1 fixed

Fit a gaussian to the likelihood, take the sigma values from the fit

In [ ]:
def func(x, a, x0, sigma): 
    return a*np.exp(-((x-x0)/sigma)**2)

parameters, covariance = curve_fit(func, omega_m_arr, likelihood_arr_fixed_w,p0=[1,best_omega_m_lambda,d_omega_m]) 
  
fit_A = parameters[0] 
fit_B = parameters[1] 
fit_C = parameters[2] 
  
fit_y = func(omega_m_arr, fit_A, fit_B, fit_C) 

one_sig_val = parameters[2]

fig,ax = plt.subplots(figsize=[8,6])

ax.plot(omega_m_arr,likelihood_arr_fixed_w,color='black',label=r"Likelihood($\Omega_m$)")
# ax.plot(omega_m_arr, fit_y, color='blue',label="Model fit",ls='-.',alpha=1)
ax.grid()
ax.set_xlim(min(omega_m_arr),max(omega_m_arr))
ax.set_ylim(0,1.05)
ax.set_title(r"Likelihood over $\Omega_m$ with w=-1")
ax.set_xticks(np.arange(min(omega_m_arr),max(omega_m_arr),0.02))
ax.set_xlabel(r'$\Omega_m$')
ax.set_ylabel('Likelihood')

ax.axvline(best_omega_m_lambda,ls='--',color='red',label=r'$max(L(\Omega_m))$='+"{:.3f}".format(best_omega_m_lambda))

ax.axvline(best_omega_m_lambda - one_sig_val,ls='-.',color='red',label=r'My 1-$\sigma=$'+"{:.3f}".format(one_sig_val))
ax.axvline(best_omega_m_lambda + one_sig_val,ls='-.',color='red')

ax.axvline(best_omega_m_lambda - 2*one_sig_val,ls='dotted',color='red',label=r'My 2-$\sigma=$'+"{:.3f}".format(2*one_sig_val))
ax.axvline(best_omega_m_lambda + 2*one_sig_val,ls='dotted',color='red')

ax.axvline(pantheon_omega_m,ls='--',color='blue',label=r"PANTHEON $\Omega_m$ = {:0.3f}".format(pantheon_omega_m))

# ax.axvline(pantheon_omega_m + pantheon_omega_m_sigma,ls='-.',color='blue',label=r"PANTHEON $\sigma$ = {:0.4f}".format(pantheon_omega_m_sigma))
# ax.axvline(pantheon_omega_m - pantheon_omega_m_sigma,ls='-.',color='blue')

# ax.axvline(pantheon_omega_m + 2*pantheon_omega_m_sigma,ls='-.',color='blue',label=r"PANTHEON $\sigma$ = {:0.4f}".format(pantheon_omega_m_sigma))
# ax.axvline(pantheon_omega_m - 2*pantheon_omega_m_sigma,ls='-.',color='blue')

for val in [-1,1]:
    if val==1:
        ax.axvline(best_omega_m_lambda+unmarginalized[0]*val,label=r"Fisher 1-$\sigma$ limit",ls='-.',c='green')
    else:
        ax.axvline(best_omega_m_lambda+unmarginalized[0]*val,ls='-.',c='green')

plt.legend()

fig.tight_layout()

plt.savefig(os.getcwd()+"/OneDimLikelihood_fixed_w_"+end_string,dpi=180)

plt.show()

print("Maximizing the likelihood:")
result_txt = r'w ={:0.4f}'.format(best_omega_m_lambda)
display(Math(result_txt))

sigma_txt = r'\sigma_{w} ='+' {:0.4f}'.format(one_sig_val)
display(Math(sigma_txt))

print("My measurement of omega_m is {:.2f} sigma away from the PANTHEON value of omega_m".format( comparison(best_omega_m_lambda,pantheon_omega_m,one_sig_val)))

### For $\Omega_m$=0.3

Fit a gaussian to the likelihood, take the sigma values from the fit

In [ ]:
parameters, covariance = curve_fit(func, w_arr, likelihood_arr_fixed_omega_m,p0=[1,best_w_lambda,d_w]) 
  
fit_A = parameters[0] 
fit_B = parameters[1] 
fit_C = parameters[2]
  
fit_y = func(w_arr, fit_A, fit_B, fit_C) 

one_sig_val = parameters[2]

fig,ax = plt.subplots(figsize=[8,6])

ax.plot(w_arr,likelihood_arr_fixed_omega_m,color='black',label=r"Likelihood(w)")
# ax.plot(w_arr, fit_y, color='blue',label="Model fit",ls='-.',alpha=1)
ax.grid()
ax.set_xlim(min(w_arr),max(w_arr))
ax.set_ylim(0,1.05)
ax.set_title(r"Likelihood over w with $\Omega_m$=0.3")
ax.set_xticks(np.arange(min(w_arr),max(w_arr),0.2))
ax.set_xlabel(r'w')
ax.set_ylabel('Likelihood')

ax.axvline(best_w_lambda,ls='--',color='red',label=r'$max(L(w))$='+"{:.3f}".format(best_w_lambda))

ax.axvline(best_w_lambda - one_sig_val,ls='-.',color='red',label=r'1-$\sigma=$'+"{:.3f}".format(one_sig_val))
ax.axvline(best_w_lambda + one_sig_val,ls='-.',color='red')

ax.axvline(best_w_lambda - 2*one_sig_val,ls='dotted',color='red',label=r'2-$\sigma=$'+"{:.3f}".format(2*one_sig_val))
ax.axvline(best_w_lambda + 2*one_sig_val,ls='dotted',color='red')

ax.axvline(pantheon_w,ls='--',color='blue',label=r'PANTHEON w='+"{:.2f}".format(pantheon_w))

# ax.axvline(pantheon_w + pantheon_w_sigma,ls='-.',color='blue',label=r'PANTHEON $\sigma$='+"{:.3f}".format(pantheon_w_sigma))
# ax.axvline(pantheon_w - pantheon_w_sigma,ls='-.',color='blue')

# ax.axvline(pantheon_w + 2*pantheon_w_sigma,ls='-.',color='blue',label=r'PANTHEON $2\sigma$='+"{:.3f}".format(2*pantheon_w_sigma))
# ax.axvline(pantheon_w - 2*pantheon_w_sigma,ls='-.',color='blue')

for val in [-1,1]:
    if val==1:
        ax.axvline(best_w_lambda+unmarginalized[1]*val,label=r"Fisher 1-$\sigma$ limit",ls='-.',c='green')
    else:
        ax.axvline(best_w_lambda+unmarginalized[1]*val,ls='-.',c='green')

plt.legend()

fig.tight_layout()

plt.savefig(os.getcwd()+"/OneDimLikelihood_fixed_omega_m_"+end_string,dpi=180)

plt.show()

print("Maximizing the likelihood:")
result_txt = r'w ={:0.4f}'.format(best_w_lambda)
display(Math(result_txt))

sigma_txt = r'\sigma_{w} ='+' {:0.4f}'.format(one_sig_val)
display(Math(sigma_txt))

print("My measurement of w is {:.2f} sigma away from the PANTHEON value of w".format( comparison(best_w_lambda,pantheon_w,one_sig_val)))